<a href="https://colab.research.google.com/github/Rifat951/Depression-detection-using-Twitter-Dataset/blob/main/AML_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re


In [4]:
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.1 MB/s 


In [5]:
import ftfy

In [7]:
np.random.seed(1234)

DEPRES_NROWS = 3200  # number of rows to read from DEPRESSIVE_TWEETS_CSV
RANDOM_NROWS = 12000 # number of rows to read from RANDOM_TWEETS_CSV
MAX_SEQUENCE_LENGTH = 140 # Max tweet size
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300
TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.2
LEARNING_RATE = 0.1
EPOCHS= 10

In [10]:
df_processd = pd.read_csv("/content/twitter_processed_data.csv", sep = '|', header = None, nrows = DEPRES_NROWS)

##Display data

In [12]:
df_processd.head(5)

,0
0,989292962323615744|2018-04-25|23:59:57|Eastern...
1,989292959844663296|2018-04-25|23:59:56|Eastern...
2,989292951716155392|2018-04-25|23:59:54|Eastern...
3,989292873664393218|2018-04-25|23:59:35|Eastern...
4,989292856119472128|2018-04-25|23:59:31|Eastern...


In [16]:
#naming the column
df_processd.columns = ["Text"]

In [15]:
df_processd.head()

,Text
0,989292962323615744|2018-04-25|23:59:57|Eastern...
1,989292959844663296|2018-04-25|23:59:56|Eastern...
2,989292951716155392|2018-04-25|23:59:54|Eastern...
3,989292873664393218|2018-04-25|23:59:35|Eastern...
4,989292856119472128|2018-04-25|23:59:31|Eastern...


In [30]:
## split the column into sub-columns and visualize the data properly

df_processed_col = df_processd.Text.str.split(pat='|',n = 6, expand=True)
df_processed_col.head(10)

,0,1,2,3,4,5,6
0,989292962323615744,2018-04-25,23:59:57,Eastern Standard Time,whosalli,The lack of this understanding is a small but ...,"1|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,"
1,989292959844663296,2018-04-25,23:59:56,Eastern Standard Time,estermnunes,i just told my parents about my depression and...,"1|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,"
2,989292951716155392,2018-04-25,23:59:54,Eastern Standard Time,TheAlphaAries,depression is something i don't speak about ev...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"
3,989292873664393218,2018-04-25,23:59:35,Eastern Standard Time,_ojhodgson,Made myself a tortilla filled with pb&j. My de...,"1|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,"
4,989292856119472128,2018-04-25,23:59:31,Eastern Standard Time,DMiller96371630,@WorldofOutlaws I am gonna need depression med...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"
5,989292843125616641,2018-04-25,23:59:28,Eastern Standard Time,PHOTOGPRODIG,my anxiety and my depression fighting over who...,"0|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,"
6,989291000911642625,2018-04-25,23:52:09,Eastern Standard Time,sp4cecounty,wow she's suddenly cured my depression and gav...,"0|2|9|,,,,,,,,,,,,,,,,,,,,,,,,,"
7,989290997346643968,2018-04-25,23:52:08,Eastern Standard Time,sassi_klassi,"I am officially done with @kanyewest. him, the...","0|0|0|,,,,,,,,,,,,,,,,,,,,,"
8,989290972973600770,2018-04-25,23:52:02,Eastern Standard Time,therealazukaobi,Me: what's wrong?My girl: *looks up at me with...,"0|0|2|#examseason,,,,,,,,,,,,,,,,,,,,,,,,,"
9,989290951012044800,2018-04-25,23:51:57,Eastern Standard Time,KateSusabu,@AusBorderForce @PeterDutton_MP @shanebazzi Ag...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,"


In [31]:
df_processed_col.columns = ["Twitter_Id", "Date", "Time", "Time_Zone", "User_Id", "Tweet", "Rating"]

In [32]:
df_processed_col.head()

,Twitter_Id,Date,Time,Time_Zone,User_Id,Tweet,Rating
0,989292962323615744,2018-04-25,23:59:57,Eastern Standard Time,whosalli,The lack of this understanding is a small but ...,"1|0|3|,,,,,,,,,,,,,,,,,,,,,,,,,"
1,989292959844663296,2018-04-25,23:59:56,Eastern Standard Time,estermnunes,i just told my parents about my depression and...,"1|0|2|,,,,,,,,,,,,,,,,,,,,,,,,,"
2,989292951716155392,2018-04-25,23:59:54,Eastern Standard Time,TheAlphaAries,depression is something i don't speak about ev...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"
3,989292873664393218,2018-04-25,23:59:35,Eastern Standard Time,_ojhodgson,Made myself a tortilla filled with pb&j. My de...,"1|0|0|,,,,,,,,,,,,,,,,,,,,,,,,,"
4,989292856119472128,2018-04-25,23:59:31,Eastern Standard Time,DMiller96371630,@WorldofOutlaws I am gonna need depression med...,"0|0|0|,,,,,,,,,,,,,,,,,,,,,,,,"


In [ ]:
#drop some columns with person info and which are not required to give data more clarity